In [1]:
import pandas as pd
from pymongo import MongoClient
import random

from lib.mongo_aggregation_verbs import *

mongo_client = MongoClient('this_mongo')
database_reference = mongo_client.twitter

In [2]:
database_reference.collection_names()

collection_reference = database_reference.vineland_test_group

test_group = database_reference.vineland_test_group

In [3]:
match_empty_url_arrays = { MATCH : { "entities.urls" : []}}

list(test_group.aggregate(
    [
        match_empty_url_arrays,
        { COUNT : "text"}
    ]
))

[{'text': 5900}]

In [4]:
job_hashtags = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
match_not_in_bad = { MATCH : { "text" : { "$in" : job_hashtags + location_hashtags}}}
project_to_text_keep_id = { PROJECT : { "text" : "$entities.hashtags.text" } }
project_to_id = { PROJECT : { "_id" : 1 } }

bad_ids = list(test_group.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_keep_id,
        unwind_text,
        project_to_lower,
        match_not_in_bad,
        project_to_id
    ]
))
bad_ids[:10], len(bad_ids)

([{'_id': ObjectId('5a7b4792829158001301258e')},
  {'_id': ObjectId('5a7b4792829158001301258e')},
  {'_id': ObjectId('5a7b4792829158001301258e')},
  {'_id': ObjectId('5a78cba2829158001300a4be')},
  {'_id': ObjectId('5a78cba2829158001300a4be')},
  {'_id': ObjectId('5a7888d48291580013009411')},
  {'_id': ObjectId('5a7888d48291580013009411')},
  {'_id': ObjectId('5a7888d48291580013009411')},
  {'_id': ObjectId('5a81aece82915800130262de')},
  {'_id': ObjectId('5a81aece82915800130262de')}],
 944)

In [5]:
bad_ids = [bad_id['_id'] for bad_id in bad_ids]
bad_ids[:10]

[ObjectId('5a7b4792829158001301258e'),
 ObjectId('5a7b4792829158001301258e'),
 ObjectId('5a7b4792829158001301258e'),
 ObjectId('5a78cba2829158001300a4be'),
 ObjectId('5a78cba2829158001300a4be'),
 ObjectId('5a7888d48291580013009411'),
 ObjectId('5a7888d48291580013009411'),
 ObjectId('5a7888d48291580013009411'),
 ObjectId('5a81aece82915800130262de'),
 ObjectId('5a81aece82915800130262de')]

In [76]:
not_in_bad_ids_and_no_url = { "_id" : {"$nin" : bad_ids}, "entities.urls" : []}
just_the_text = {"text" : 1, "created_at" : 1, "_id" : 0}

In [77]:
test_group.find_one(
    not_in_bad_ids_and_no_url,
    just_the_text
)

{'created_at': 'Sat Feb 03 23:19:15 +0000 2018',
 'text': '@curlyxmichelle Suck to be you they’re all mine now!'}

In [78]:
cur  = test_group.find(
    not_in_bad_ids_and_no_url,
    just_the_text
)

tweets = list(cur)
tweet_text = pd.DataFrame(tweets)

In [79]:
len(tweet_text)

5900

In [80]:
tweet_text.head()

,created_at,text
0,Sat Feb 03 23:19:15 +0000 2018,@curlyxmichelle Suck to be you they’re all min...
1,Sat Feb 03 23:20:37 +0000 2018,@SkizzyMars I like every song you drop
2,Sat Feb 03 23:21:00 +0000 2018,@__beniHANA Throw everything away minus the coat
3,Sat Feb 03 23:22:45 +0000 2018,My girlfriend and I were freaking out. Awesome...
4,Sat Feb 03 23:23:54 +0000 2018,@PamelaM5050 @jtimberlake @ladygaga That's awe...


In [98]:
tweet_text['day'] = tweet_text['created_at'].str[:3]
tweet_text['month'] = tweet_text['created_at'].str[4:8]
tweet_text['date'] = tweet_text['created_at'].str[8:11]
tweet_text['hour'] = tweet_text['created_at'].str[11:13]
tweet_text['minute'] = tweet_text['created_at'].str[14:16]
tweet_text['second'] = tweet_text['created_at'].str[17:19]
tweet_text['year'] = tweet_text['created_at'].str[-4:]

AttributeError: 'Series' object has no attribute 'to_numeric'

In [108]:
tweet_text.hour = pd.to_numeric(tweet_text.hour)

In [100]:
tweet_text.text = tweet_text.text.str.replace('http\S+|www.\S+', '', case=False)

In [101]:
tweet_text[tweet_text['text'].str.contains("trump")]

,created_at,text,day,month,date,hour,minute,second,year
391,Sun Feb 04 13:40:42 +0000 2018,@jonnywoo34 What trumpsters don’t seem to comp...,Sun,Feb,04,13,40,42,2018
431,Sun Feb 04 14:53:03 +0000 2018,@FrogDirt @jamie0219 @FoxNews @KellyannePolls ...,Sun,Feb,04,14,53,03,2018
661,Sun Feb 04 19:37:03 +0000 2018,@SpeakerRyan These trump voting morons try to ...,Sun,Feb,04,19,37,03,2018
757,Sun Feb 04 21:32:15 +0000 2018,@MarkDanielsPJ They must work for trump😃🎱,Sun,Feb,04,21,32,15,2018
1770,Mon Feb 05 07:47:18 +0000 2018,"@texastrumper - Thanks for the ""like."" Be fre...",Mon,Feb,05,07,47,18,2018
1873,Mon Feb 05 14:25:12 +0000 2018,"@ericbolling @realDonaldTrump Eric, you'll be ...",Mon,Feb,05,14,25,12,2018
2154,Mon Feb 05 22:32:07 +0000 2018,@artislifeisart @DNCpeep it's BECAUSE we love ...,Mon,Feb,05,22,32,07,2018
2260,Tue Feb 06 02:40:36 +0000 2018,@matthew_melton @vicky4trump @RepAdamSchiff @P...,Tue,Feb,06,02,40,36,2018
3403,Wed Feb 07 22:05:44 +0000 2018,@Scavino45 Quit using my tax dollars to publis...,Wed,Feb,07,22,05,44,2018
4207,Sat Feb 10 01:24:10 +0000 2018,@1SnoozyQ Sensitive information that proves tr...,Sat,Feb,10,01,24,10,2018


In [84]:
tweet_text[tweet_text['text'].str.contains("Trump")]

,created_at,text,day,month,date,hour,minute,second,year
34,Sun Feb 04 00:05:09 +0000 2018,@BorisEP @billmaher @RealTimers @davidfrum @re...,Sun,Feb,04,00,05,09,2018
48,Sun Feb 04 00:20:48 +0000 2018,@JrcheneyJohn @DrLee4America @realDonaldTrump ...,Sun,Feb,04,00,20,48,2018
95,Sun Feb 04 01:16:25 +0000 2018,@waltshaub @realDonaldTrump Shaubie-doobie-doo..,Sun,Feb,04,01,16,25,2018
126,Sun Feb 04 01:46:31 +0000 2018,"@realDonaldTrump Also, Mr Pres , can Mueller, ...",Sun,Feb,04,01,46,31,2018
246,Sun Feb 04 04:26:26 +0000 2018,@joncoopertweets Wrong in Russia there will be...,Sun,Feb,04,04,26,26,2018
257,Sun Feb 04 04:37:52 +0000 2018,Trump has failed to uphold his oath to Protect...,Sun,Feb,04,04,37,52,2018
325,Sun Feb 04 07:08:50 +0000 2018,@IWV @POTUS @realDonaldTrump @WhiteHouse They ...,Sun,Feb,04,07,08,50,2018
350,Sun Feb 04 10:52:39 +0000 2018,@GartrellLinda @TrumpsDC Freakish evil bitch,Sun,Feb,04,10,52,39,2018
411,Sun Feb 04 14:08:07 +0000 2018,@realDonaldTrump @POTUS The Nunes Memo vindica...,Sun,Feb,04,14,08,07,2018
418,Sun Feb 04 14:20:37 +0000 2018,"@JCBwellUS Or the Huckster’s-same empty space,...",Sun,Feb,04,14,20,37,2018


In [85]:
tweet_text[tweet_text['text'].str.contains("eaglestweet_text[tweet_text['text'].str.contains("Trump")]")]

,created_at,text,day,month,date,hour,minute,second,year
20,Sat Feb 03 23:47:36 +0000 2018,So glad my favorite eagles player of all time ...,Sat,Feb,03,23,47,36,2018
215,Sun Feb 04 03:46:15 +0000 2018,"I'm gonna cry whether the eagles win or lose, ...",Sun,Feb,04,03,46,15,2018
459,Sun Feb 04 15:28:16 +0000 2018,"@mreeseeagles I hope you have a great call, Me...",Sun,Feb,04,15,28,16,2018
690,Sun Feb 04 20:12:21 +0000 2018,@JonesyAustin25 I’ll root for anyone over the ...,Sun,Feb,04,20,12,21,2018
859,Sun Feb 04 23:13:28 +0000 2018,Fly eagles fly on the road to victory E.A.G.L....,Sun,Feb,04,23,13,28,2018
864,Sun Feb 04 23:14:52 +0000 2018,@TheWetBandits19 Me too fuck the eagles. More ...,Sun,Feb,04,23,14,52,2018
880,Sun Feb 04 23:24:30 +0000 2018,@MsVivicaFox @Eagles I’m rooting for the eagle...,Sun,Feb,04,23,24,30,2018
1001,Mon Feb 05 00:25:33 +0000 2018,"As a Ravens fan, I’m cheering on Torrey Smith ...",Mon,Feb,05,00,25,33,2018
1060,Mon Feb 05 00:46:35 +0000 2018,Never will you see me rooting for the eagles e...,Mon,Feb,05,00,46,35,2018
1290,Mon Feb 05 02:19:06 +0000 2018,I’m not a eagles fan but I NEVER seen so much ...,Mon,Feb,05,02,19,06,2018


In [86]:
tweet_text[tweet_text['text'].str.contains("Eagles")]

,created_at,text,day,month,date,hour,minute,second,year
215,Sun Feb 04 03:46:15 +0000 2018,"I'm gonna cry whether the eagles win or lose, ...",Sun,Feb,04,03,46,15,2018
417,Sun Feb 04 14:19:42 +0000 2018,@Eagles 🏈💚🏈🦅🤩❤️❤️❤️😥😭😉,Sun,Feb,04,14,19,42,2018
476,Sun Feb 04 15:53:08 +0000 2018,Wesson is all ready to cheer\n#FlyEagleFly #Go...,Sun,Feb,04,15,53,08,2018
498,Sun Feb 04 16:21:16 +0000 2018,@TonyStewart @SuperBowl Eagles by 2 scores,Sun,Feb,04,16,21,16,2018
508,Sun Feb 04 16:35:49 +0000 2018,‘Just got back from @Wawa ! ‘Got my free Eagle...,Sun,Feb,04,16,35,49,2018
605,Sun Feb 04 18:32:36 +0000 2018,#FlyEaglesFly 🐶🦅💚 https://t.co/2TzMAQIW81,Sun,Feb,04,18,32,36,2018
663,Sun Feb 04 19:39:42 +0000 2018,@pacojara12 @maxkellerman Yes the Eagles are a...,Sun,Feb,04,19,39,42,2018
679,Sun Feb 04 19:57:50 +0000 2018,Eagles gotta win this for meek,Sun,Feb,04,19,57,50,2018
697,Sun Feb 04 20:21:23 +0000 2018,The only dub Eagles are taking tonight will be...,Sun,Feb,04,20,21,23,2018
729,Sun Feb 04 20:54:19 +0000 2018,Eagles all the wayyy🦅💚,Sun,Feb,04,20,54,19,2018


In [87]:
tweet_text[tweet_text['text'].str.contains("love")]

,created_at,text,day,month,date,hour,minute,second,year
3,Sat Feb 03 23:22:45 +0000 2018,My girlfriend and I were freaking out. Awesome...,Sat,Feb,03,23,22,45,2018
4,Sat Feb 03 23:23:54 +0000 2018,@PamelaM5050 @jtimberlake @ladygaga That's awe...,Sat,Feb,03,23,23,54,2018
28,Sun Feb 04 00:00:05 +0000 2018,I love tax return 💰💰,Sun,Feb,04,00,00,05,2018
49,Sun Feb 04 00:27:10 +0000 2018,@SkylerrBee17 @GirIsWant 😂😂so good ❤️❤️❤️I lov...,Sun,Feb,04,00,27,10,2018
97,Sun Feb 04 01:16:55 +0000 2018,a love story https://t.co/NbPawP9OCy,Sun,Feb,04,01,16,55,2018
111,Sun Feb 04 01:36:25 +0000 2018,@Chuchotais This is true. I’m gonna instant mi...,Sun,Feb,04,01,36,25,2018
178,Sun Feb 04 03:02:08 +0000 2018,love my bff https://t.co/R9AhCk7PrX,Sun,Feb,04,03,02,08,2018
209,Sun Feb 04 03:37:40 +0000 2018,@tlcat06 @ColoradoMelodyG @ilovetalib @bronc4l...,Sun,Feb,04,03,37,40,2018
229,Sun Feb 04 04:08:37 +0000 2018,@ilovetalib @tlcat06 @elway4ever @notraidergir...,Sun,Feb,04,04,08,37,2018
233,Sun Feb 04 04:12:46 +0000 2018,@LavidaIoca Without me u suck! Imma save this ...,Sun,Feb,04,04,12,46,2018


In [180]:
def counts_by_hour(df):
    result_array = []
    for h in range(0,23):
        result = len(df[df['hour'] == h].index)
        result_array = np.append(result_array, [result], axis=0)
    return result_array

In [188]:
import numpy as np
love = counts_by_hour(tweet_text[tweet_text['text'].str.contains("love")])
trump = counts_by_hour(tweet_text[tweet_text['text'].str.contains("Trump")])
eagles = counts_by_hour(tweet_text[tweet_text['text'].str.contains("Eagles")])
total = counts_by_hour(tweet_text)
print(trump)
print(love)
print(eagles)
print(total)

[  8.  11.   3.   5.   4.   1.   0.   1.   1.   1.   1.   1.   3.   5.  16.
   6.   5.   6.   9.   7.   7.   7.   6.]
[  9.  16.   7.  10.  16.   7.   0.   1.   0.   1.  11.   6.   3.   6.   4.
  12.  10.   7.   8.   5.   7.   6.  10.]
[ 12.   8.   5.  34.   4.   2.   0.   1.   0.   0.   0.   0.   2.   1.   2.
   2.   6.   0.   2.   4.   2.   4.   5.]
[ 411.  456.  481.  612.  317.  197.  100.   57.   39.   45.   62.   97.
  145.  234.  260.  294.  240.  247.  254.  234.  235.  243.  270.]


In [169]:
len(tweet_text[tweet_text['hour'] == 3].index)

612

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

tfidf.fit(tweet_text['text'].values.astype('U'))
word_occurence = tfidf.transform(tweet_text.text)

In [17]:
words = tfidf.get_feature_names()
word_sample = random.sample(words, 20)
word_occurence_m = pd.DataFrame(word_occurence, columns=words)
word_occurence_m[word_sample].head()

PandasError: DataFrame constructor not properly called!

In [18]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=10)
lda.fit(word_occurence)

lda_df = pd.DataFrame(lda.components_, columns=words).T

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [19]:
def filter_topic(lda_df, index, threshold):
    return (lda_df[lda_df[index] > threshold][index]
            .sort_values(ascending=False))

In [20]:
filter_topic(lda_df, 0, 15)

happy        35.535929
love         31.678952
day          24.589881
moves        20.098288
valentine    19.283183
birthday     18.719289
thing        15.605242
man          15.137371
Name: 0, dtype: float64

In [21]:
filter_topic(lda_df, 1, 15)

did     22.970579
ass     17.959398
wait    15.856711
Name: 1, dtype: float64

In [22]:
filter_topic(lda_df, 2, 10)

snapchat    18.596315
follow      16.698565
Name: 2, dtype: float64

In [23]:
filter_topic(lda_df, 3, 10)

good       32.396095
better     26.692799
need       23.431256
getting    19.986087
oh         17.736814
night      15.979918
work       15.841187
great      15.426505
wanna      14.854182
just       14.820676
tonight    13.659379
wow        13.079137
game       12.988591
life       12.833985
god        12.678332
day        11.614904
year       11.244192
sleep      11.194827
wants      11.129253
gt         10.416270
like       10.280626
makes      10.019820
Name: 3, dtype: float64

In [28]:
filter_topic(lda_df, 4, 10)

big     12.260592
mood    10.722284
Name: 4, dtype: float64

In [25]:
filter_topic(lda_df, 5, 25)

don       34.017434
eagles    32.398904
know      29.502571
just      27.757399
want      26.626931
time      25.549081
Name: 5, dtype: float64

In [30]:
filter_topic(lda_df, 6, 10)

thanks    25.883177
fuck      21.551408
yes       15.941914
baby      14.431280
pretty    10.830471
person    10.048215
Name: 6, dtype: float64

In [31]:
filter_topic(lda_df, 7, 10)

black    12.571580
guess    10.365215
bitch    10.303794
Name: 7, dtype: float64

In [32]:
filter_topic(lda_df, 8, 10)

feel               18.661185
realdonaldtrump    17.813945
best               16.204613
like               14.128831
hate               13.073524
morning            12.894555
does               11.942656
bro                11.008773
start              10.309576
Name: 8, dtype: float64

In [ ]:
filter_topic(lda_df, 9, 10)